# Домашнее задание по теме: Регрессия (линейная/полиномиальная) с использованием `Pipeline`

## Подготовка окружения

In [ ]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [ ]:
# !conda install pandas matplotlib numpy seaborn scikit-learn -y

## **ВНИМАНИЕ**

- **Задания описаны кратко, только самое необходимое, в основном только идеи и подсказки. Это творческая работа.**

- Для выполнения заданий можно использовать любые удобные способы, если в комментариях к заданию не указан конкретный метод. 

- Желательно использовать максимум пройденной информации или более продвинутые способы (если нашли самостоятельно).

- Желательно писать свои идеи/мысли по поводу методов/результатов.

- В строке "Подсказка" могут быть перечислены какие-то методы, которыми можно воспользоваться, но это не обязательно, так как это только один из множества вариантов (не факт, что лучший).

- Обязательно **использовать `Pipeline`**.

- Графики можно рисовать любым способом / цветом и т.п. Главное, чтобы было понятно, что там изображено.

- При выполнении задания необходимо понимать, что значит каждое слово (на английском). Если значение слова непонятно, то необходимо разобраться с его значением самостоятельно. Пожалуйста, будьте внимательны, перепроверяйте значения слов, если не уверены.

## Импорт необходимых пакетов

In [ ]:
import numpy as np

np.__version__

In [ ]:
import pandas as pd

pd.__version__

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.__version__

In [ ]:
import seaborn as sns

sns.__version__

In [ ]:
import sklearn
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

sklearn.__version__

## Загрузка данных

[Источник (Medical Cost Personal Datasets)](https://www.kaggle.com/mirichoi0218/insurance)

In [ ]:
df = pd.read_csv("./../../data/insurance.csv")

df

In [ ]:
df.info()

## Преобразование типов

Привести в порядок типы данных. В результате не должно быть типа `object`.

In [ ]:
df['sex'] = df.sex.astype('category')
df['smoker'] = df.smoker.astype('category')
df['region'] = df.region.astype('category')

In [ ]:
df.info()

## Разделить на train/test

- Разделить данные на `y` и `X`. 
- `y` - это колонка `charges`.
- Разделить данные на train / test.
- **В дальнейшем при анализе использовать строго `train` данные и не подсматривать в `test`.**

In [ ]:
from sklearn.model_selection import train_test_split

target_col = 'charges'
y = df[target_col].copy()
X = df.drop(target_col, axis=1)

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Выбросы

- Есть ли выбросы? Каково распределение данных?
    - `hist`
    - `kde`
    - `boxplot`
- Нужно ли с этим что-то делать?

In [ ]:
df_train = X_train.join(y_train)
df_train

In [ ]:
df_num_cols = df_train.select_dtypes(include=np.number)
df_num_cols_len = len(df_num_cols.columns)
df_num_cols_len

In [ ]:
fig, axs = plt.subplots(df_num_cols_len, 1, figsize=(8, 7))

for index, num_col in enumerate(df_num_cols):
    sns.histplot(data=df_train[num_col], 
                 kde=True, 
                 ax=axs[index])
fig.tight_layout()

## Анализ корреляций

- Проверить нет ли мультиколлинеарности.
- Если есть, то нужно ли что-то с этим делать?

Подсказка: в `sns.heatmap` можно указать параметры `vmin`, `vmax` для более адекватного распределения цвета.

In [ ]:
correlation_matrix = df_train.corr()

plt.figure(figsize=(5, 5))

ax = sns.heatmap(
    correlation_matrix,
    square=True,
    annot=True,
    fmt='.2f',
    cmap='PRGn',
    cbar_kws={"shrink": .5},
    annot_kws={"size": 12},
    robust=True,
    vmin=-1, 
    vmax=1
)

plt.title('Correlation Matrix of features', fontsize=20)

## Полиномиальная множественная регрессия

- Можно использовать любое количество признаков.
- Попробовать несколько различных степеней полинома.
- Если необходимо, то использовать методы feature engineering.
- Помнить о том, что может быть полезна нормализация/стандартизация.
- Обучить модель линейной регрессии.
- Проанализировать результаты (различные метрики).

In [ ]:
for degree in range(2, 5):
    print(f'degree = {degree}')
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train[['age', 'bmi', 'children']])
    X_test_poly = poly.transform(X_test[['age', 'bmi', 'children']])
    poly_regr = LinearRegression()
    poly_regr.fit(X_train_poly, y_train)
    
    y_train_predicted = poly_regr.predict(X_train_poly)
    y_test_predicted = poly_regr.predict(X_test_poly)

    print(f'R2 (train): {r2_score(y_train, y_train_predicted)}')
    print(f'R2 (test): {r2_score(y_test, y_test_predicted)}')

    print(f"MSE (train): {mean_squared_error(y_train, y_train_predicted, squared=True)}")
    print(f"MSE (test): {mean_squared_error(y_test, y_test_predicted, squared=True)}")
    
    print('-' * 15, '\n')
